
# RNN using LSTM 
       




<img src="img/RNN-rolled.png"/ width="80px" height="80px">

<img src="img/RNN-unrolled.png"/ width="400px" height="400px">

<img src="img/LSTM3-chain.png"/ width="800px" height="800px">

_source: http://colah.github.io/posts/2015-08-Understanding-LSTMs_

In [1]:
from keras.optimizers import SGD
from keras.preprocessing.text import one_hot,text_to_word_sequence,base_filter
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence

Using Theano backend.


In [2]:
from sklearn.cross_validation import train_test_split

In [3]:
import os
import pickle
import numpy as np

In [4]:
import pandas as pd

In [5]:
DATA_DIRECTORY = os.path.join('data')
print DATA_DIRECTORY

data


In [7]:
with open(os.path.join(DATA_DIRECTORY,"male_blog_list.txt"),"rb") as male_file:
    male_posts= pickle.load(male_file)
with open(os.path.join(DATA_DIRECTORY,"female_blog_list.txt"),"rb") as female_file:
    female_posts = pickle.load(female_file)

In [8]:
filtered_male_posts = []
filtered_female_posts = []

for post_male in male_posts:
    if len(post_male) == 0:
        continue
    filtered_male_posts.append(post_male)

for post_female in female_posts:
    if len(post_female) == 0:
        continue
    filtered_female_posts.append(post_female)

In [16]:
# text processing - one hot builds index of the words
male_one_hot = []
female_one_hot = []
n = 30000
for post in filtered_male_posts:
    try:
        male_one_hot.append(one_hot(post,n,split=" ",filters=base_filter(),lower=True))
    except Exception as e:
        continue

for post in filtered_female_posts:
    try:
        female_one_hot.append(one_hot(post,n,split=" ",filters=base_filter(),lower=True))
    except:
        continue

In [17]:
# 0 for male, 1 for female
concatenate_array_rnn = np.concatenate((np.zeros(len(male_one_hot)),np.ones(len(female_one_hot))))

In [18]:
x_train_rnn,x_test_rnn,y_train_rnn,y_test_rnn = train_test_split(np.concatenate((female_one_hot,male_one_hot)),concatenate_array_rnn,test_size=0.2)

In [19]:
maxlen = 100
x_train_rnn = sequence.pad_sequences(x_train_rnn,maxlen=maxlen)
x_test_rnn = sequence.pad_sequences(x_test_rnn,maxlen=maxlen)
print('x_train_rnn shape:', x_train_rnn.shape,y_train_rnn.shape)
print('x_test_rnn shape:', x_test_rnn.shape,y_test_rnn.shape)

('x_train_rnn shape:', (3706, 100), (3706,))
('x_test_rnn shape:', (927, 100), (927,))


In [20]:
max_features = 30000
dimension = 128
input_dimension = 128
output_dimension = 128


In [21]:
model = Sequential()
model.add(Embedding(max_features, dimension))
model.add(LSTM(output_dim = output_dimension, input_dim=input_dimension))
model.add(Dropout(0.5))
model.add(Dense(input_dim=output_dimension, output_dim=1))
model.add(Activation('sigmoid'))

In [22]:
model.compile(loss='mean_squared_error',optimizer='sgd')

In [23]:
model.fit(x_train_rnn,y_train_rnn,batch_size=32,nb_epoch=4,validation_data=(x_test_rnn,y_test_rnn),show_accuracy=True)

Train on 3706 samples, validate on 927 samples
Epoch 1/4
3706/3706 [==============================] - 29s - loss: 0.2487 - val_loss: 0.2478
Epoch 2/4
3706/3706 [==============================] - 31s - loss: 0.2483 - val_loss: 0.2478
Epoch 3/4
3706/3706 [==============================] - 29s - loss: 0.2485 - val_loss: 0.2477
Epoch 4/4
3706/3706 [==============================] - 30s - loss: 0.2482 - val_loss: 0.2478


/home/nischal/anaconda2/lib/python2.7/site-packages/keras/models.py:385: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


In [25]:
model.evaluate(x_test_rnn,y_test_rnn,batch_size=32)

927/927 [==============================] - 2s     


0.24775136778977053

In [28]:
predicted_output = model.predict(x_test_rnn,batch_size=32)

In [41]:
model.predict_classes(x_test_rnn,batch_size=32)

927/927 [==============================] - 2s     


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [32]:
df = pd.DataFrame(columns=['predicted','actual'])

In [34]:
print type(predicted_output)

<type 'numpy.ndarray'>


In [37]:
predicted_output.flatten()

array([ 0.47257459,  0.47150251,  0.47041783,  0.45502865,  0.46951264,
        0.46091244,  0.47031957,  0.47972605,  0.46834189,  0.46798551,
        0.47614822,  0.4678463 ,  0.47411162,  0.46905404,  0.46607849,
        0.45298687,  0.46298698,  0.47241601,  0.48014697,  0.4752118 ,
        0.47300324,  0.47477165,  0.44004026,  0.46681237,  0.44474387,
        0.44107056,  0.47893703,  0.47737437,  0.46906811,  0.46242693,
        0.47132847,  0.46405166,  0.48006848,  0.45297685,  0.4537138 ,
        0.4697125 ,  0.46210769,  0.44221985,  0.45534465,  0.47295734,
        0.45190829,  0.45475361,  0.45834431,  0.4756121 ,  0.45683247,
        0.45522338,  0.45910046,  0.45680383,  0.47381619,  0.44358069,
        0.45045453,  0.47051814,  0.47065336,  0.47720304,  0.46651417,
        0.46838152,  0.46061912,  0.44572857,  0.46910205,  0.46858227,
        0.46960133,  0.47160026,  0.47484818,  0.48020723,  0.47740191,
        0.46048611,  0.4555622 ,  0.46555412,  0.48442936,  0.47

In [38]:
df['predicted'] = predicted_output.flatten()

In [39]:
df['actual'] = y_test_rnn

In [40]:
df.head()

,predicted,actual
0,0.472575,0
1,0.471503,1
2,0.470418,0
3,0.455029,1
4,0.469513,0


# Using TFIDF Vectorizer as an input instead of one hot

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [10]:
vectorizer = TfidfVectorizer(decode_error='ignore', norm='l2')
tfidf_male = vectorizer.fit_transform(filtered_male_posts)
tfidf_female = vectorizer.fit_transform(filtered_female_posts)

In [11]:
flattened_array_tfidf_male = tfidf_male.toarray()
flattened_array_tfidf_female = tfidf_male.toarray()

In [12]:
concatenate_array_rnn = np.concatenate((np.zeros(len(flattened_array_tfidf_male)),np.ones(len(flattened_array_tfidf_female))))

In [13]:
x_train_rnn,x_test_rnn,y_train_rnn,y_test_rnn = train_test_split(np.concatenate((flattened_array_tfidf_male,flattened_array_tfidf_female)),concatenate_array_rnn,test_size=0.2)

In [14]:
maxlen = 100
# x_train_rnn = sequence.pad_sequences(x_train_rnn,maxlen=maxlen)
# x_test_rnn = sequence.pad_sequences(x_test_rnn,maxlen=maxlen)
# print('x_train_rnn shape:', x_train_rnn.shape,y_train_rnn.shape)
# print('x_test_rnn shape:', x_test_rnn.shape,y_test_rnn.shape)

In [16]:
max_features = 30000
dimension = 128
input_dimension = 128
output_dimension = 128

In [17]:
model = Sequential()
model.add(Embedding(max_features, dimension))
model.add(LSTM(output_dim = output_dimension, input_dim=input_dimension))
model.add(Dropout(0.5))
model.add(Dense(input_dim=output_dimension, output_dim=1))
model.add(Activation('sigmoid'))

In [18]:
model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
model.fit(x_train_rnn,y_train_rnn,batch_size=32,nb_epoch=4,validation_data=(x_test_rnn,y_test_rnn),show_accuracy=True)

In [ ]:
score,acc = model.evaluate(x_test_rnn,y_test_rnn,batch_size=32,show_accuracy=True)

# Sentence Generation using RNN(LSTM)

In [ ]:
# reading all the male text data into one string
male_post = ' '.join(filtered_male_posts[:2])

#building character set for the male posts
character_set_male = set(male_post)
#building two indices - character index and index of character
char_indices = dict((c, i) for i, c in enumerate(character_set_male))
indices_char = dict((i, c) for i, c in enumerate(character_set_male))


# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 1
sentences = []
next_chars = []
for i in range(0, len(male_post) - maxlen, step):
    sentences.append(male_post[i : i + maxlen])
    next_chars.append(male_post[i + maxlen])


In [ ]:
#Vectorisation of input
x_male = np.zeros((len(male_post),maxlen,len(character_set_male)),dtype=np.bool)
y_male = np.zeros((len(male_post),len(character_set_male)),dtype=np.bool)

print x_male.shape,y_male.shape

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_male[i, t, char_indices[char]] = 1
    y_male[i, char_indices[next_chars[i]]] = 1

print x_male.shape,y_male.shape

In [ ]:

#Building the model to generate text with 2 layers
auto_text_generating_male_model = Sequential()
auto_text_generating_male_model.add(LSTM(len(character_set_male),512,return_sequences=True))
auto_text_generating_male_model.add(Dropout(0.2))
auto_text_generating_male_model.add(LSTM(512,512,return_sequences=False))
auto_text_generating_male_model.add(Dropout(0.2))
auto_text_generating_male_model.add(Dense(512,len(character_set_male)))
auto_text_generating_male_model.add(Activation('sigmoid'))

In [ ]:
auto_text_generating_male_model.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
import random,sys

In [ ]:
# helper function to sample an index from a probability array
def sample(a, diversity=0.75):
    if random.random() > diversity:
        return np.argmax(a)
    while 1:
        i = random.randint(0, len(a)-1)
        if a[i] > random.random():
            return i

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1,10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    auto_text_generating_male_model.fit(x_male, y_male, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(male_post) - maxlen - 1)

    for diversity in [0.2, 0.4, 0.6, 0.8]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = male_post[start_index : start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')

        for iteration in range(400):
            try:
                x = np.zeros((1, maxlen, len(character_set_male)))
                for t, char in enumerate(sentence):
                    x[0, t, char_indices[char]] = 1.

                preds = auto_text_generating_male_model.predict(x, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                #sys.stdout.write(next_char)
                #sys.stdout.flush()
            except:
                continue
                
        print sentence
        print()